# Part 1

In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import numpy as np

In [2]:
request = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
bs = BeautifulSoup(request.content,'lxml')
table = bs.find_all('table')[0]
temp_df = pd.read_html(str(table))
df=pd.DataFrame(temp_df[0])

The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [3]:
df=df[df.Borough != 'Not assigned']
df.reset_index(inplace=True)
df.drop('index', axis=1, inplace=True)
df.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor


If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.

In [4]:
idx_list = df.loc[df['Neighbourhood'] == "Not assigned"].index
for i in idx_list:
    df.iloc[i, 2] = df.iloc[i, 1] 
df.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor


More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

In [5]:
df=df.groupby(['Postcode','Borough'])['Neighbourhood'].apply(list)
df=df.sample(frac=1).reset_index()
df['Neighbourhood']=df['Neighbourhood'].str.join(',')
df.head()

,Postcode,Borough,Neighbourhood
0,M2P,North York,York Mills West
1,M1N,Scarborough,"Birch Cliff,Cliffside West"
2,M1W,Scarborough,L'Amoreaux West
3,M4H,East York,Thorncliffe Park
4,M3L,North York,Downsview West


In [6]:
df.shape

(103, 3)